In [1]:
import numpy as np
import os
import pandas as pd
import glob
import h5py
import tables
from ast import literal_eval

## Merging Data

This script takes raw data, create a pandas df and saves it to disk as a hdf5 file called `data.h5`. Before running this code, make sure your file directory looks like this...

```
.
│   Data Merged.ipynb
│   ...    
│
└───Data
    │   search_grid.csv
    │   impact_score_2017.csv
    │   impact_score_2017.csv
    |   impact_score_2019_public_test.csv
    |
    └───GFS_2017
    |   │   gfs_4_20170101_0000_000.csv
    |   │   ...
    |
    └───GFS_2018
    |   │   gfs_4_20180101_0000_000.csv
    |   │   ...     
    | 
    └───GFS_2019
        │   gfs_4_20190101_0000_000.csv
        │   ...     
   

```

In [2]:
#### Path to all data files

# All weather
weather17 = ["./Data/GFS_2017/" + f for f in os.listdir("./Data/GFS_2017")]
weather18 = ["./Data/GFS_2018/" + f for f in os.listdir("./Data/GFS_2018")]
weather19 = ["./Data/GFS_2019/" + f for f in os.listdir("./Data/GFS_2019")]
allweather = weather17 + weather18 + weather19

# All impact
allimpact = ["./Data/" + f for f in os.listdir("./Data") if f.startswith('impact')]

# Loc2zip map
grid = "./Data/search_grid.csv"

In [3]:
### Load weather into 1 df
# Concat into a giant dataframe
weather = pd.concat(map(pd.read_csv, allweather), ignore_index=True)

In [4]:
### Load and flatten loc2zip map
loc2zip = pd.read_csv(grid, converters={"mapped_zipcodes": literal_eval})
loc2zip = loc2zip.explode("mapped_zipcodes").reset_index()
loc2zip['mapped_zipcodes'] = pd.to_numeric(loc2zip['mapped_zipcodes'])
#loc2zip.rename(columns={"mapped_zipcodes":"zip5"}, inplace=True)

In [5]:
### Load impact into 1 df
impact = pd.concat(map(pd.read_csv, allimpact), ignore_index=True).drop(["Unnamed: 0"], axis=1)

In [6]:
### Join impact and loc2zip
impact_loc = impact.join(loc2zip.set_index('mapped_zipcodes'), on='zip5').drop(["index"], axis = 1)

In [7]:
### Join impact and weather in loc and date
# Preprocessing on join columns 
impact_loc["date_key"] = pd.to_datetime(impact_loc["date_key"])
weather["Date"] = pd.to_datetime(weather["Date"], format="%Y%m%d")

data = impact_loc.merge(weather, left_on=["date_key", "grid_lat", "grid_lon"], right_on=["Date", "lat", "lng"])
data = data.sort_values(by="Date")

In [8]:
### Save data as hdf5 file
data.to_hdf("data.h5", key='df', mode='a')